In [1]:
import torch
import time

# Tutorial: Higher Order Array Manipulation

For the sake of simplicity, we define $n$-th order tensors as arrays of dimension $n$. A $0$-th order array is a scalar, a $1$-st order array is a vector in $\mathbb{R}^{d_1}$, and a $2$-nd order array is a matrix in $\mathbb{R}^{d_1\times d_2}$. Going further, a $n$-th order array is an element of $\mathbb{R}^{d_1\times...\times d_n}$ for some dimensions $(d_i)_{i\in[n]}$.

## Declaration

In [2]:
# Declare a third order array 
d1, d2, d3 = 2, 3, 5
A = torch.rand(d1, d2, d3)

print("The shape of A is {}".format(list(A.shape)))

The shape of A is [2, 3, 5]


## Indexing

Say we have a $3$-rd order array $\mathbf{A}\in\mathbb{R}^{d_1\times d_2\times d_3}$. Indexing and slicing works as for lower order arrays:

In [3]:
print("A[0] has shape {}".format(list(A[0].shape)))
print("A[:, 1:, :] has shape {}".format(list(A[:, 1:, :].shape)))
print("A[:, 1, 2:4] has shape {}".format(list(A[:, 1, 2:4].shape)))

A[0] has shape [3, 5]
A[:, 1:, :] has shape [2, 2, 5]
A[:, 1, 2:4] has shape [2, 2]


We can also use a different indexing array $\mathbf{b}$ to index $\mathbf{A}$. This indexing operates on the first dimension of $\mathbf{A}$, meaning that if $\mathbf{b}\in\mathbb{R}^{l_1\times l_2}$, then `A[b]` will have shape $l_1\times l_2\times d_2\times d_3$.

In [4]:
b = torch.tensor([0, 0, 1, 0])
print("If A has shape {}, b has shape {}, then A[b] has shape {}.".format(list(A.shape), list(b.shape), list(A[b].shape)))

b = torch.tensor([[0, 0, 1, 0], [1, 1, 0, 1]])
print("If A has shape {}, b has shape {}, then A[b] has shape {}.".format(list(A.shape), list(b.shape), list(A[b].shape)))

If A has shape [2, 3, 5], b has shape [4], then A[b] has shape [4, 3, 5].
If A has shape [2, 3, 5], b has shape [2, 4], then A[b] has shape [2, 4, 3, 5].


... This works provided the indexing array $\mathbf{b}$ has integer values comprised between $0$ and $d_{1}-1$ (included).

In [5]:
try:
    b = torch.tensor([0, 0, 2, 0])
    A[b]
except Exception as e:
    print("We have an out-of bound indexing: d_1=1 but max b=2")
    print("The exception is: {}".format(e))

We have an out-of bound indexing: d_1=1 but max b=2
The exception is: index 2 is out of bounds for dimension 0 with size 2


## Operations

Imagine now that we have a batch of $1000$ $d\times d$ matrices: $(\mathbf{a}_i)_{i\in[1000]}$, for which we want to compute the trace. We could loop over the matrices and compute the traces separately.

In [6]:
d = 2
attempts = 10

ais    = [torch.rand(d, d) for i in range(10000)]

start  = time.time()
for _ in range(attempts):
    traces = ...
end    = time.time()

print("Elapsed time: {:.2e}s.".format((end - start) / attempts))

Elapsed time: 1.13e-02s.


Alternatively, we could vectorize this operation using a three dimensional array $\mathbf{A}\in\mathbb{R}^{1000\times d\times d}$ that contains the stacked matrices.

In [7]:
A = torch.stack(ais, axis=0)

print("A has shape {}".format(list(A.shape)))

start  = time.time()
for _ in range(attempts):
    traces = ...
end    = time.time()

print("Elapsed time: {:.2e}s.".format((end - start) / attempts))

A has shape [10000, 2, 2]
Elapsed time: 2.68e-04s.


And we reduced the computation time by one or two order(s) of magnitude! A different option that we will use extensively during part 3 is to use [Einstein summation](https://en.wikipedia.org/wiki/Einstein_notation). For the traces computation this is written as: $\mathbf{A}_{i,j,j}$. This can be done with PyTorch with the method [`torch.einsum`](https://pytorch.org/docs/stable/generated/torch.einsum.html).

In [8]:
start  = time.time()
for _ in range(attempts):
    traces = ...
end    = time.time()

print("Elapsed time: {:.2e}s.".format((end - start) / attempts))

Elapsed time: 1.15e-04s.


As efficient as the trace method! Also, `torch.einsum` is highly flexible. It can compute the transpose of a batch of arrays, or various kinds of matrix multiplications.

In [9]:
ais = [torch.rand(2, 3) for i in range(1000)]

# Transpose each stacked matrices
A = torch.stack(ais, axis=0)
print("A has shape {}".format(list(A.shape)))
AT = ...
print("A^T has shape {}".format(list(AT.shape)))

A has shape [1000, 2, 3]
A^T has shape [1000, 3, 2]


Below we show how to compute $\mathbf{a}_i^\top\mathbf{a}_i$ for some matrices $\mathbf{a}_i\in\mathbb{R}^{2\times 3}$ using `torch.einsum`.

In [10]:
print("A has shape {}".format(list(A.shape)))
product_As = ...
print("Stacked ai^T.ai has shape {}".format(list(product_As.shape)))

A has shape [1000, 2, 3]
Stacked ai^T.ai has shape [1000, 3, 3]


To scale each $\mathbf{a}_i$ by a weight $w_i$, we can still use `torch.einsum`. Define the vector containing all the weights $\mathbf{w}=(w_i)_i$, we have:

In [11]:
w = torch.rand(A.shape[0])
weighted_A = ...
print("Weighted and stacked ai has shape {}".format(list(weighted_A.shape)))

Weighted and stacked ai has shape [1000, 2, 3]
